### Генерация поэзии с помощью нейронных сетей: шаг 1
##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

Ваша основная задача: научиться генерироват стихи с помощью простой рекуррентной нейронной сети (Vanilla RNN). В качестве корпуса текстов для обучения будет выступать роман в стихах "Евгений Онегин" Александра Сергеевича Пушкина.

In [15]:
# do not change the code in the block below
# __________start of block__________
import string
import os
from random import sample

import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F

from IPython.display import clear_output

import matplotlib.pyplot as plt
# __________end of block__________

In [16]:
# do not change the code in the block below
# __________start of block__________
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('{} device is available'.format(device))
# __________end of block__________

cuda device is available


#### 1. Загрузка данных.

In [17]:
# do not change the code in the block below
# __________start of block__________
# !curl -O https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt
    
with open('onegin.txt', 'r', encoding='utf-8') as iofile:
    text = iofile.readlines()
    
text = "".join([x.replace('\t\t', '').lower() for x in text])
# __________end of block__________

#### 2. Построение словаря и предобработка текста
В данном задании требуется построить языковую модель на уровне символов. Приведем весь текст к нижнему регистру и построим словарь из всех символов в доступном корпусе текстов. Также добавим токен `<sos>`.

In [18]:
# do not change the code in the block below
# __________start of block__________
tokens = sorted(set(text.lower())) + ['<sos>']
num_tokens = len(tokens)

assert num_tokens == 84, "Check the tokenization process"

token_to_idx = {x: idx for idx, x in enumerate(tokens)}
idx_to_token = {idx: x for idx, x in enumerate(tokens)}

assert len(tokens) == len(token_to_idx), "Mapping should be unique"

print("Seems fine!")


text_encoded = [token_to_idx[x] for x in text]
# __________end of block__________

Seems fine!


__Ваша задача__: обучить классическую рекуррентную нейронную сеть (Vanilla RNN) предсказывать следующий символ на полученном корпусе текстов и сгенерировать последовательность длины 100 для фиксированной начальной фразы.

Вы можете воспользоваться кодом с занятие №6 или же обратиться к следующим ссылкам:
* Замечательная статья за авторством Andrej Karpathy об использовании RNN: [link](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* Пример char-rnn от Andrej Karpathy: [github repo](https://github.com/karpathy/char-rnn)
* Замечательный пример генерации поэзии Шекспира: [github repo](https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb)

Данное задание является достаточно творческим. Не страшно, если поначалу оно вызывает затруднения. Последняя ссылка в списке выше может быть особенно полезна в данном случае.

Далее для вашего удобства реализована функция, которая генерирует случайный батч размера `batch_size` из строк длиной `seq_length`. Вы можете использовать его при обучении модели.

In [19]:
# do not change the code in the block below
# __________start of block__________
batch_size = 256
seq_length = 100
start_column = np.zeros((batch_size, 1), dtype=int) + token_to_idx['<sos>']

def generate_chunk():
    global text_encoded, start_column, batch_size, seq_length

    start_index = np.random.randint(0, len(text_encoded) - batch_size*seq_length - 1)
    data = np.array(text_encoded[start_index:start_index + batch_size*seq_length]).reshape((batch_size, -1))
    yield np.hstack((start_column, data))
# __________end of block__________    

Пример батча:

In [20]:
next(generate_chunk())

array([[83,  5,  0, ...,  1, 76,  1],
       [83, 47, 50, ..., 61,  1, 59],
       [83, 62, 63, ...,  1, 59, 46],
       ...,
       [83, 47, 45, ..., 57,  1, 69],
       [83, 45, 63, ..., 64, 62, 55],
       [83, 45, 50, ..., 63,  5,  1]])

Далее вам предстоит написать код для обучения модели и генерации текста.

In [44]:
class CharRNN(nn.Module):
    def __init__(self, num_tokens, hidden_size, num_layers=2, dropout=0.3):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embed = nn.Embedding(num_tokens, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers,
                            dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_tokens)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        output, hidden = self.lstm(x, hidden)
        logits = self.fc(output)
        return logits, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters())
        return (weight.new_zeros(self.num_layers, batch_size, self.hidden_size),
                weight.new_zeros(self.num_layers, batch_size, self.hidden_size))


В качестве иллюстрации ниже доступен график значений функции потерь, построенный в ходе обучения авторской сети (сам код для ее обучения вам и предстоит написать).

In [47]:
hidden_size = 256
num_layers = 2
dropout = 0.3

model = CharRNN(num_tokens=num_tokens, hidden_size=hidden_size,
                num_layers=num_layers, dropout=dropout).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
criterion = nn.CrossEntropyLoss()




In [45]:
def train(model, epochs=20, patience=3):
    best_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for _ in range(100):
            batch = next(generate_chunk())
            inputs = torch.tensor(batch[:, :-1], dtype=torch.long).to(device)
            targets = torch.tensor(batch[:, 1:], dtype=torch.long).to(device)

            hidden = model.init_hidden(inputs.size(0))  # batch_size
            hidden = tuple(h.to(device) for h in hidden)

            optimizer.zero_grad()
            outputs, _ = model(inputs, hidden)
            loss = criterion(outputs.view(-1, num_tokens), targets.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / 100
        print(f"Epoch {epoch + 1}: loss = {avg_loss:.4f}")

        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch + 1}")
                break


Шаблон функции `generate_sample` также доступен ниже. Вы можете как дозаполнить его, так и написать свою собственную функцию с нуля. Не забывайте, что все примеры в обучающей выборке начинались с токена `<sos>`.

In [46]:
def generate_sample(model, seed_phrase=None, max_length=200, temperature=1.0, device=device):
    model.eval()

    if seed_phrase:
        x_sequence = [token_to_idx['<sos>']] + [token_to_idx[c] for c in seed_phrase]
    else:
        x_sequence = [token_to_idx['<sos>']]

    input_seq = torch.tensor([[x_sequence[0]]], dtype=torch.long).to(device)
    hidden = model.init_hidden(1)
    hidden = tuple(h.to(device) for h in hidden)

    generated = []

    # прогоняем seed_phrase
    for idx in x_sequence:
        input_seq = torch.tensor([[idx]], dtype=torch.long).to(device)
        output, hidden = model(input_seq, hidden)

    idx = x_sequence[-1]

    for _ in range(max_length - len(x_sequence)):
        input_seq = torch.tensor([[idx]], dtype=torch.long).to(device)
        output, hidden = model(input_seq, hidden)
        output = output[:, -1, :] / temperature
        probabilities = F.softmax(output, dim=-1).detach().cpu().numpy().flatten()
        idx = np.random.choice(len(probabilities), p=probabilities)
        generated.append(idx)

    full_sequence = x_sequence + generated
    return ''.join([idx_to_token[i] for i in full_sequence if idx_to_token[i] != '<sos>'])


In [48]:
train(model, epochs=50, patience=5)


Epoch 1: loss = 2.6093
Epoch 2: loss = 2.0562
Epoch 3: loss = 1.8490
Epoch 4: loss = 1.6954
Epoch 5: loss = 1.5841
Epoch 6: loss = 1.4786
Epoch 7: loss = 1.3757
Epoch 8: loss = 1.3038
Epoch 9: loss = 1.2056
Epoch 10: loss = 1.1472
Epoch 11: loss = 1.1086
Epoch 12: loss = 1.0487
Epoch 13: loss = 0.9914
Epoch 14: loss = 0.9341
Epoch 15: loss = 0.9195
Epoch 16: loss = 0.8985
Epoch 17: loss = 0.8584
Epoch 18: loss = 0.8627
Epoch 19: loss = 0.8059
Epoch 20: loss = 0.7969
Epoch 21: loss = 0.7745
Epoch 22: loss = 0.7578
Epoch 23: loss = 0.7436
Epoch 24: loss = 0.7250
Epoch 25: loss = 0.7143
Epoch 26: loss = 0.6837
Epoch 27: loss = 0.6928
Epoch 28: loss = 0.6981
Epoch 29: loss = 0.6650
Epoch 30: loss = 0.6474
Epoch 31: loss = 0.6692
Epoch 32: loss = 0.6617
Epoch 33: loss = 0.6672
Epoch 34: loss = 0.6307
Epoch 35: loss = 0.6179
Epoch 36: loss = 0.6201
Epoch 37: loss = 0.6212
Epoch 38: loss = 0.5927
Epoch 39: loss = 0.6082
Epoch 40: loss = 0.6217
Epoch 41: loss = 0.5985
Epoch 42: loss = 0.5901
E

In [49]:
seed_phrase = ' мой дядя самых честных правил'

generated_phrases = [
    generate_sample(
        model,
        seed_phrase,
        max_length=500,
        temperature=0.8  # можешь поэкспериментировать: 0.7–1.2
    ).replace('<sos>', '')
    for _ in range(10)
]


Пример текста сгенерированного обученной моделью доступен ниже. Не страшно, что в тексте много несуществующих слов. Используемая модель очень проста: это простая классическая RNN.

In [52]:
print(generate_sample(model, ' мой дядя самых честных правил\n', max_length=2000, temperature=0.4))

 мой дядя самых честных правил


vi

в том соседственных селений
ему не нравились пиры;
бежал он их беседы шумной,
их разговор благоразумный
о сенокосе, о вине,
о псарне, о своей родне,
конечно, не блистал ни чувством,
ни поэтическим огнем,
ни остротою, ни умом,
ни общежития искусством;
но от них оставляет отдохнуть.



xlvi

дай оглянусь. простите ж, сени,
где дни мои текли в глуши,
испрожен душою погружена,
в сем сердце, полном одета счастья,
хранит безмолвно между тем;
так будто разрочь у риссотами;
но разговор их милых жен
гораздо меньше был умен.



xii

богат, хорош собою, ленский
везде был принят как жених;
таков обычай деревенский;
все дочек прочили своих
за полурусского соседа;
взойдет ли он, тотчас беседа
заводит слово стороной
о скуке жизни холостой;
зовут соседа к самовару,
а дету самого утратя
приготовляясь, денег развел,
онегин рыхлый и брань,
любви мечтательницы милой,
но где его твой страстных стрела.



xxxvi

и так они старели оба.
и отворились наконец
перед супругом 

### Сдача задания
Сгенерируйте десять последовательностей длиной 500, используя строку ' мой дядя самых честных правил'. Температуру для генерации выберите самостоятельно на основании визуального качества генериуремого текста. Не забудьте удалить все технические токены в случае их наличия.

Сгенерированную последовательность сохрание в переменную `generated_phrase` и сдайте сгенерированный ниже файл в контест.

In [27]:
seed_phrase = ' мой дядя самых честных правил'

In [28]:
generated_phrases = # your code here

# For example:

# generated_phrases = [
#     generate_sample(
#         model,
#         ' мой дядя самых честных правил',
#         max_length=500,
#         temperature=1.
#     ).replace('<sos>', '')
#     for _ in range(10)
# ]

SyntaxError: invalid syntax (3861625285.py, line 1)

In [ ]:
# do not change the code in the block below
# __________start of block__________

import json
if 'generated_phrases' not in locals():
    raise ValueError("Please, save generated phrases to `generated_phrases` variable")

for phrase in generated_phrases:

    if not isinstance(phrase, str):
        raise ValueError("The generated phrase should be a string")

    if len(phrase) != 500:
        raise ValueError("The `generated_phrase` length should be equal to 500")

    assert all([x in set(tokens) for x in set(list(phrase))]), 'Unknown tokens detected, check your submission!'
    

submission_dict = {
    'token_to_idx': token_to_idx,
    'generated_phrases': generated_phrases
}

with open('submission_dict.json', 'w') as iofile:
    json.dump(submission_dict, iofile)
print('File saved to `submission_dict.json`')
# __________end of block__________

На этом задание завершено. Поздравляем!